# **Finding the SharePoint Integration Feature Dependent Power BI Reports in a Tenant**

# Check out this [blog post](https://bits2bi.com/2025/10/08/finding-the-hidden-connections-power-bi-reports-born-in-sharepoint/) for detailed context about this notebook

##### **Credits**
The code used for making Power BI REST API calls, processing, storing JSON results, etc. is adapted from FUAM (Fabric Unified Admin Monitoring). Big thanks to their awesome developers! Check out the FUAM [here](https://github.com/microsoft/fabric-toolbox/tree/main/monitoring/fabric-unified-admin-monitoring).

##### **Permissions**
The user running this notebook must be a **Fabric administrator**.

##### **Instructions**

- Attach a Lakehouse to the notebook
- Update the variables in the Input Section
- Run the notebook

**⚠️ Note:** The code assumes that only Power BI workspaces created through the SharePoint integration feature have descriptions starting with "Sharepoint".
If you have other workspaces with descriptions that also start with "Sharepoint", review the results carefully and exclude those from the final output as needed.

## Prepare the environment

In [ ]:
# Import necessary libraries
from sempy import fabric
from notebookutils.mssparkutils.credentials import getToken

import pandas as pd
import numpy as np
import requests
import json

import datetime # For adding timesteamps
import time # For using sleep()


In [ ]:
# <--------------------- I N P U T --------------------- S E C T I O N --------------------->

# ---- SET THE write_to_files TO FALSE IF YOU DO NOT WANT THE SCANNER API RESULTS TO BE STORED IN THE LAKEHOUSE ---- #
write_to_files = True

# ---- SET THE write_the_processed_df_as_delta_table TO FALSE IF YOU DO NOT WANT THE PROCESSED DATAFRAME TO BE STORED AS A DELTA TABLE ---- #
write_the_processed_df_as_delta_table = True

# ---- SET THE delta_table_name FOR YOUR TARGET DELTA TABLE ---- #
delta_table_name = "sharepoint_integration_feature_pbi_reports"

In [ ]:
## Variables

# Scanner API request specific
workspaces_per_request = 100
max_parallel_requests = 16

# Set date helpers
current_time = datetime.datetime.now()

# Init the REST client
client = fabric.FabricRestClient()

# Array of scan results
results = []

## Power BI REST API calls

### Helper functions for REST API calls

In [ ]:
def GenerateHeader():
    # Retrieve an access token for the logged-in user using mssparkutils
    # This token is scoped for Power BI ("pbi") and is used for authenticated API calls
    # Reference: https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-utilities#get-token
    access_token = getToken("pbi")

    # Construct the HTTP headers required for API requests
    # - 'Content-Type': specifies the format of the request body as JSON
    # - 'Authorization': includes the Bearer token for secure access
    header = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    # Return the prepared headers for use in API calls
    return header

In [ ]:
def RequestWithRetry(
    method, url, data={}, num_retries=3, success_list=[200, 202, 404], **kwargs
):
    # Generate authorization headers for the request
    headers = GenerateHeader()
    response = None

    # Attempt the request up to `num_retries` times
    for i in range(num_retries):
        try:
            # Make the appropriate HTTP request based on the method
            if method == "post":
                response = client.post(url, json=data, headers=headers, **kwargs)
            if method == "get":
                response = client.get(url, headers=headers, **kwargs)

            # If the response status code is in the success list, return the response
            if response.status_code in success_list:
                return response

            # Handle rate limiting (HTTP 429 Too Many Requests)
            if response.status_code == 429:
                retry_after = response.headers.get("Retry-After", None)

                # Case 1: Hit the 500 requests per hour limit
                if retry_after is not None:
                    waitTime = int(retry_after)
                    print(
                        f"Hit the 500 requests per hour rate limit - waiting {waitTime} seconds until next retry"
                    )
                    time.sleep(waitTime)

                # Case 2: Hit the 16 simultaneous requests limit (no Retry-After header)
                else:
                    waitTime = 120
                    print(
                        f"Hit the 16 simultaneous requests limit - waiting {waitTime} seconds until next retry"
                    )
                    time.sleep(waitTime)

        except Exception as e:
            # Log any unexpected exceptions and continue retrying
            print(e)
            pass

    # Return the last response (could be None or an error response)
    return response

### Get the list of workspaces

**API Documentation:** https://learn.microsoft.com/en-us/rest/api/power-bi/admin/groups-get-groups-as-admin

In [ ]:
# Send a GET request to the Power BI Admin API to retrieve up to 5000 workspaces
# Filter only those whose description starts with 'Sharepoint'

response = RequestWithRetry(
    "get",
    "v1.0/myorg/admin/groups?$top=5000&$filter=startswith(description,'Sharepoint')",
)

# Normalize the 'value' field from the JSON response into a flat DataFrame
# This makes nested JSON data easier to work with in tabular format
filtered_workspaces = pd.json_normalize(response.json()["value"])

filtered_workspaces_count = len(filtered_workspaces)

# Print the total number of filtered workspaces
print("Total workspaces:", filtered_workspaces_count)

# Display a count of workspaces grouped by their description
# Useful for understanding how many workspaces fall under each SharePoint category
print("Count by description:")
display(
    filtered_workspaces.groupby("description", as_index=False)["id"]
    .count()
    .rename(columns={"id": "count"})  # Rename for clarity
)

# Display the full filtered workspace DataFrame for inspection or further analysis
display(filtered_workspaces)

### Batch the list of workspaces for Scanner API calls

In [ ]:
# Add a numeric index column to the DataFrame for grouping
filtered_workspaces["index"] = pd.to_numeric(filtered_workspaces.index)

# Calculate the 'run' group by dividing the index by the number of workspaces per request
# This helps in batching workspaces into manageable chunks for Scanner API calls
filtered_workspaces["run"] = filtered_workspaces["index"] // workspaces_per_request

# Group workspace IDs by 'run' and convert them into lists
# Each group (run) will contain a list of workspace IDs to be processed together
filtered_workspaces = filtered_workspaces.groupby("run")["id"].apply(list)

# Initialize a new DataFrame to track the status of each run
df_runs = pd.DataFrame(data=filtered_workspaces)

# Add a status column to monitor progress of each run (e.g., API call status)
df_runs["status"] = "Not Started"

### Scanner APIs to get the artifact level details for the workspaces

**API Documentation:**
- https://learn.microsoft.com/en-us/rest/api/power-bi/admin/workspace-info-post-workspace-info
- https://learn.microsoft.com/en-us/rest/api/power-bi/admin/workspace-info-get-scan-status
- https://learn.microsoft.com/en-us/rest/api/power-bi/admin/workspace-info-get-scan-result

In [ ]:
# Filter runs that are either not started, already requested, or currently running
# Limit to a maximum number of parallel requests (e.g., 16)
df_runs_current = df_runs[
    df_runs["status"].isin(["Not Started", "Request sent", "Running"])
].head(max_parallel_requests)

# Continue processing while there are runs in progress or pending
while df_runs_current.shape[0] > 0:
    # Wait briefly before checking statuses or sending new requests
    time.sleep(5)

    # Iterate through each run in the current batch
    for i, row in df_runs_current.iterrows():
        # If the run hasn't started yet, send a POST request to initiate metadata generation
        if row["status"] == "Not Started":
            payload = {"workspaces": row["id"]}
            powerBIAPIBaseUri = "https://api.powerbi.com/v1.0/myorg/"
            api_uri = f"{powerBIAPIBaseUri}/admin/workspaces/getInfo?lineage=true&datasourceDetails=true"

            # Send the request with retry logic to handle rate limits and transient errors
            response = RequestWithRetry("post", api_uri, payload)

            # Extract the run ID from the response and update the status
            id = pd.json_normalize(response.json())["id"][0]
            df_runs.loc[i, "status"] = "Request sent"
            df_runs.loc[i, "run_id"] = id

        # If the run is already requested or running, check its current status
        elif row["status"] in ["Request sent", "Running"]:
            response = RequestWithRetry(
                "get", f"/v1.0/myorg/admin/workspaces/scanStatus/{row['run_id']}"
            )

            # Extract and update the latest status from the response
            stat = pd.json_normalize(response.json())["status"][0]
            df_runs.loc[i, "status"] = stat

    # Refresh the list of runs that are still in progress or pending
    df_runs_current = df_runs[
        df_runs["status"].isin(["Not Started", "Request sent", "Running"])
    ].head(max_parallel_requests)

In [ ]:
print ("Scan status of each batch:")
df_runs

### Parsing and writing the Scanner API results

In [ ]:
# Iterate through each scan run in the tracking DataFrame
for i, row in df_runs.iterrows():
    # Proceed only if the scan run has completed successfully
    if row["status"] == "Succeeded":
        # Retrieve the scan result using the run_id
        response = RequestWithRetry(
            "get", f"/v1.0/myorg/admin/workspaces/scanResult/" + row["run_id"]
        )

        # Append the JSON result to the results list for further processing or analysis
        results.append(response.json())

        # If writing to files is enabled, save the result as a JSON file
        if write_to_files:
            # Construct the folder path using current date and time for organized storage
            folder_path = (
                mssparkutils.fs.getMountPath('/default')
                + "/Files/SharePoint_Integration_Inventory/"
                + current_time.strftime("%Y/%m/%d")
                + "/"
                + current_time.strftime("%H-%M-%S")
                + "/"
            )

            # Create the directory if it doesn't exist
            mssparkutils.fs.mkdirs("file://" + folder_path)

            # Write the scan result to a JSON file named after the run_id
            with open(folder_path + row["run_id"] + ".json", "w") as f:
                f.write(json.dumps(response.json()))  # Fixed variable name from `response_json` to `response.json()`
                print(
                    "/v1.0/myorg/admin/workspaces/scanResult/"
                    + row["run_id"]
                    + ": written to lakehouse"
                )

## Find the SharePoint site connection details for the workspaces

In [ ]:
# Initialize lists to collect flattened data
all_reports = []
all_datasets = []
all_datasources = []

# Iterate through each entry in the results list
for entry in results:
    # Collect all datasource instances
    all_datasources.extend(entry.get('datasourceInstances', []))
    
    # Iterate through each workspace in the entry
    for ws in entry.get('workspaces', []):
        ws_id = ws['id']
        ws_name = ws['name']
        
        # Flatten and collect report data
        for report in ws.get('reports', []):
            report_flat = {
                'workspace_id': ws_id,
                'workspace_name': ws_name,
                **report
            }
            all_reports.append(report_flat)
        
        # Flatten and collect dataset data
        for dataset in ws.get('datasets', []):
            dataset_flat = {
                'workspace_id': ws_id,
                'workspace_name': ws_name,
                **dataset
            }
            all_datasets.append(dataset_flat)

# Convert collected lists to DataFrames
reports_df = pd.DataFrame(all_reports)
datasets_df = pd.DataFrame(all_datasets)
datasource_df = pd.DataFrame(all_datasources)

# --- Flatten nested structures ---

# Explode the 'datasourceUsages' list so each usage becomes a separate row
datasets_exploded = datasets_df.explode('datasourceUsages')

# Normalize the JSON structure inside 'datasourceUsages'
datasource_details = pd.json_normalize(datasets_exploded['datasourceUsages'])

# Combine normalized datasource details with the exploded dataset DataFrame
datasets_flattened = pd.concat(
    [datasets_exploded.drop(columns=['datasourceUsages']), datasource_details],
    axis=1
)

# Normalize the JSON structure inside 'connectionDetails' from datasource instances
connection_details_df = pd.json_normalize(datasource_df['connectionDetails'])

# Combine normalized connection details with the original datasource DataFrame
datasource_flattened = pd.concat(
    [datasource_df.drop(columns=['connectionDetails']), connection_details_df],
    axis=1
)

# --- Join datasets with reports and datasource info ---

# Step 1: Join datasets with reports using workspace_id and datasetId
reports_datasets_joined = pd.merge(
    datasets_flattened[['workspace_id', 'workspace_name', 'id', 'name', 'datasourceInstanceId']],
    reports_df[['workspace_id', 'workspace_name', 'id', 'name', 'datasetId']],
    left_on=['workspace_id', 'id'],
    right_on=['workspace_id', 'datasetId'],
    how='left',
    suffixes=('_dataset', '_report')
)

# Step 2: Join the result with datasource details using datasourceInstanceId
joined_df = pd.merge(
    reports_datasets_joined,
    datasource_flattened[['datasourceId', 'datasourceType', 'sharePointSiteUrl']],
    left_on='datasourceInstanceId',
    right_on='datasourceId',
    how='left'
)

# --- Final cleanup and column renaming ---

# Select and rename relevant columns for clarity
joined_df = joined_df[[
    'workspace_id',
    'workspace_name_dataset',
    'id_report',
    'name_report',
    'id_dataset',
    'name_dataset',
    'datasourceType',
    'sharePointSiteUrl'
]].rename(columns={
    'workspace_name_dataset': 'workspace_name',
    'id_report': 'report_id',
    'name_report': 'report_name',
    'id_dataset': 'semantic_model_id',
    'name_dataset': 'semantic_model_name'
})

# Add a derived column to classify SharePoint site type
joined_df['sharePointSiteType'] = np.where(
    joined_df['sharePointSiteUrl'].str.contains('personal', case=False, na=False),
    'Personal',
    'Team'
)
# Extract SharePoint site owner from the URL if it contains 'personal'
joined_df['sharePointSiteOwner'] = joined_df['sharePointSiteUrl'].apply(
    lambda url: url.split('/')[-1] if pd.notnull(url) and 'personal' in url.lower() else None
)

## Write the processed data to delta table

In [ ]:
if write_the_processed_df_as_delta_table:
    print(f"Writing the dataframe to delta table: {delta_table_name}")

    # Convert the Pandas DataFrame to a Spark DataFrame
    spark_df = spark.createDataFrame(joined_df)

    # Write the Spark DataFrame to a Delta table
    # - mode("overwrite"): replaces the existing table if it exists
    # - option("overwriteSchema", "true"): updates the schema if it has changed
    # - format("delta"): uses Delta Lake format for ACID transactions and versioning
    # - saveAsTable(): saves the data as a managed table in the metastore
    spark_df.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable(delta_table_name)

## Conclusions

In [ ]:
print("--------- SUMMARY ---------")

print("Total workspaces:", filtered_workspaces_count)
print("Workspaces with Power BI items:", joined_df["workspace_id"].nunique())
print(f"Workspaces without any Power BI items: {filtered_workspaces_count - joined_df['workspace_id'].nunique()}")
print("Total distinct semantic models:", joined_df["semantic_model_id"].nunique())
print("Total distinct reports:", joined_df["report_id"].nunique())

print("Workspaces by SharePoint Site Type:")
display(
    joined_df.groupby("sharePointSiteType", as_index=False)["workspace_id"]
    .nunique()
    .rename(columns={"workspace_id": "count"})
)

print("--------- DETAILS ---------")
display(joined_df)